In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import imageio as io
from ipywidgets import interact
from perlin_numpy import generate_perlin_noise_2d
from scipy.ndimage import binary_dilation, binary_erosion
from skimage.draw import disk
from skimage.filters import sobel

2022-06-22 12:12:34.642881: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Segmentation Models: using `tf.keras` framework.


In [2]:
tf.executing_eagerly()

True

Check Perlin noise

In [4]:
@interact
def testInteract(i:(0,5,1)=1, t:(0.8,1,.01)=0):
    np.random.seed(0)
    n = generate_perlin_noise_2d((256, 256), (2**i,2**i))
    
    n = (n - n.min())/(n.max()-n.min())
    plt.imshow(n > t)
    plt.colorbar()

interactive(children=(IntSlider(value=1, description='i', max=5), FloatSlider(value=0.8, description='t', max=…

Creating a fast IoU computation function

In [ ]:
im = io.v3.imread("training/0.png")
seg = io.v3.imread("training/0_seg.png")

plt.imshow(seg)

In [9]:
def fast_iou(gt, pr, eps=1e-7):
    a = gt.astype(bool)
    b = pr.astype(bool)
    
    i = a * b
    u = a + b
    
    iou = np.count_nonzero(i) / (np.count_nonzero(u) + eps)
    
    return iou

In [10]:
fast_iou(seg,seg)

0.9999999997409326

Creating Training Pairs using some hyperparameters

In [ ]:
def createTrainingPair(seg, p_dilation=0.1, p_erosion=0.1, p_noise=0.1, p_disk=0.1, p_contour=0.1):
    new_seg = seg.copy() > 0
    
    if np.random.uniform() <= p_contour:
        seg_edges = sobel(seg)
        seg_edges = binary_dilation(seg_edges)
        new_seg[seg_edges.astype(np.float32) * np.random.uniform() > np.random.uniform()] = np.random.choice([True, False])
        
    
    if np.random.uniform() <= p_disk:
        x = np.asarray(np.where(new_seg))
        
        if x.shape[1]:
            for _ in range(np.random.randint(1,5)):
                ix = np.random.randint(0, x.shape[1])

                try:
                    rr, cc = disk(x[:, ix], np.random.uniform(1,3), shape=new_seg.shape)

                    new_seg[rr,cc] = np.random.choice([True, False])
                except:
                    pass

    if np.random.uniform() <= p_dilation:
        # print("Dilate...")
        for _ in range(np.random.randint(1,5)):
            new_seg = binary_dilation(new_seg)

    if np.random.uniform() <= p_erosion:
        # print("Erode...")
        for _ in range(np.random.randint(1,5)):
            new_seg = binary_erosion(new_seg)
            
    if np.random.uniform() <= p_noise:
        # print("Perlin noise...")
        i = np.random.randint(0, 4)
        t = np.random.uniform(0.9, 0.99)
        
        n = generate_perlin_noise_2d(seg.shape, (2**i,2**i))
        n = (n - n.min())/(n.max()-n.min())
        
        new_seg[n > t] = np.random.choice([True, False])
        
    # print("IoU score...")
    y = fast_iou(seg, new_seg)
    
    # print("Done!")
    
    return new_seg, y

Test the function and show random samples

In [73]:
new_seg, y = createTrainingPair(seg, 0,0,0,0,1.)

In [67]:
@interact
def showRandomSamples(i:(0,10)=0):
    new_seg, y = createTrainingPair(seg, .7, .7, .3, .1)
    
    plt.imshow(new_seg)
    plt.title(f"IoU: {y}")

interactive(children=(IntSlider(value=0, description='i', max=10), Output()), _dom_classes=('widget-interact',…